# IBM Ponder This - October 2024

## Problem Statement

We are looking for a natural number X satisfying the following properties of its decimal representation:

1. All the digits \{1,2,3,4,6,7,8,9\} appear at least once, but \{0, 5\} do not appear at all.

2. X can be written as a front part A and back part B, such that if X were a string, it could be written as A concatenated with B, such that

B is the product of the digits of X.
X is a multiple of B.
B is a perfect square.
For example, X=3411296 has the property that its digit product is 1296 so it can be split into A=341 and B=1296 such that properties 2.1 and 2.3 are satisfied. However, none of the other properties are satisfied in this case since X is missing the digits 7,8 and is not a multiple of B .


Your goal: Find the least number X satisfying the above properties.

A bonus "*" will be given for finding the least number X satisfying the above properties except property 2.3 which is replaced by a new property: 2.3 The digit product of A is a perfect cube.

## Solution

One strategy is to generate candidates for $B$ and, for each valid candidate, attempt to construct $X$.

We utilize a modified version of the algorithm for finding the [$n$-th Ugly Number](https://leetcode.com/problems/ugly-number-ii/editorial/), which traditionally generates numbers with prime factors 2, 3, and 5. In our case, we focus on the factors 2, 3, and 7, suitable for constructing numbers from the digits $\{1,2,3,4,6,7,8,9\}$. Each candidate number $B$ is generated sequentially by multiplying the smallest previously found number by 2, 3, or 7, respectively. Additionally, we keep track of the multiplicity of each factor used to generate the candidates.

### Base Problem

For each candidate number $B$ generated, we ensure it is a perfect square and contains no digits 0 or 5. Additionally, we adjust the total count of each factor by subtracting the contributions from each digit in $B$ (e.g., the digit 8 contributes three factors of 2, so we subtract three from the count of 2). This helps in determining if the remaining factors are sufficient and non-negative to potentially form $A$, ensuring that all required digits can appear in $X$. Whenever the multiplicity of a factor becomes negative while removing the contributions from the digits of $B$, we exclude the candidate.

We generate a list of candidates for $B$ (we arbitrarily set the maximum size of the list and can adjust if no solution is found) in this way and keep track of the remaining factors that can be used to create the digits of $A$. Then for each candidate $B$, we generate all the potential $A$. This is done by creating all the digit combinations from the remaining factors and all the permutations of permutations of those digits. We also need to consider adding ones as an arbitrarily large number of ones can be present in $X$ without affecting the other conditions. To reduce the run time, we arbitrarily set a maximum number of digits for $X$ which can be adjusted upwards is no solution is found. This way, we have a maximum number of ones to add to each combination.

For each permutation, of the digits of $A$, we create $X$ and check if $X$ is divisible by $B$ and if all the digits are present is $X$. The answer to the problem is the smallest such $X$.

### Bonus Problem

The logic for the bonus problem is very similar but requires additional optimisations to obtain the solution in a reasonable time.

We generate the candidates for $B$ in a similar fashion except we remove the constraint that $B$ is square. For each candidate, we also generate all the digit combinations for $A$ from the remaining factors but this time, we check if the product of the digits is a cube. Then we form $X$ from $A$ and $B$ and verify that $X$ contains all the necessary digits. If only the digit 1 is missing, we add it to the list of digits for $A$ as this won't impact the cube condition. Then, we add the digit combination to a priority queue (min heap) where the sorting is based on the number of digits. The goal of this is to avoid unnecessarily processing large numbers if a solution with a lower number is already found. As in the base problem, we also consider combinations with added ones as long as the number of digits is lower than the maximum number of digits we set.

One we have all the digit combinations, we process them in order. For each combination, we generate all the unique permutations of digits for $A$ with a backtracking algorithm. Finally we return the permutation leading to the smaller valid $X$ divisible by $B$.

In [1]:
import math
import itertools
import heapq
from collections import Counter


candidates = []

ugly_numbers = [(72576, {2: 7, 3: 4, 7: 1})]  # Starting with 72576 and its factorization
index_2, index_3, index_7 = 0, 0, 0

# Initial multiples based on the first ugly number
next_2 = (ugly_numbers[0][0] * 2, {**ugly_numbers[0][1], 2: ugly_numbers[0][1][2] + 1})
next_3 = (ugly_numbers[0][0] * 3, {**ugly_numbers[0][1], 3: ugly_numbers[0][1][3] + 1})
next_7 = (ugly_numbers[0][0] * 7, {**ugly_numbers[0][1], 7: ugly_numbers[0][1][7] + 1})

# for _ in range(1, n):
while len(candidates) < 12:
    # Determine the minimum next ugly number
    next_ugly, factors = min([next_2, next_3, next_7], key=lambda x: x[0])

    # Append the new ugly number and its factors
    ugly_numbers.append((next_ugly, factors))
    if math.isqrt(next_ugly)**2 == next_ugly:
        s = Counter(str(next_ugly))
        if not ('0' in s or '5' in s):
            remaining = factors.copy()
            not_candidate = False
            for key, val in s.items():
                if key == '2':
                    remaining[2] -= val
                elif key == '3':
                    remaining[3] -= val
                elif key == '4':
                    remaining[2] -= 2 * val
                elif key == '6':
                    remaining[2] -= val
                    remaining[3] -= val
                elif key == '7':
                    remaining[7] -= val
                elif key == '8':
                    remaining[2] -= 3 * val
                elif key == '9':
                    remaining[3] -= 2 * val
            if remaining[2] < 0 or remaining[3] < 0 or remaining[7]< 0:
                not_candidate = True
            if not not_candidate:
                candidates.append((next_ugly, factors, remaining))

    # Update the pointers and calculate the next values
    if next_ugly == next_2[0]:
        index_2 += 1
        base_num, base_factors = ugly_numbers[index_2]
        next_2 = (base_num * 2, {**base_factors, 2: base_factors.get(2, 0) + 1})

    if next_ugly == next_3[0]:
        index_3 += 1
        base_num, base_factors = ugly_numbers[index_3]
        next_3 = (base_num * 3, {**base_factors, 3: base_factors.get(3, 0) + 1})

    if next_ugly == next_7[0]:
        index_7 += 1
        base_num, base_factors = ugly_numbers[index_7]
        next_7 = (base_num * 7, {**base_factors, 7: base_factors.get(7, 0) + 1})

In [2]:
def gen_comb(two, three, seven, path, max_len, res):
    if len(path) >= max_len:
        return
    if two == 0 and three == 0 and seven == 0:
        res.append(path[:])
        return
    if three >= 2:
        gen_comb(two, three-2, seven, path + [9], max_len, res)
    if two >= 3:
        gen_comb(two-3, three, seven, path + [8], max_len, res)
    if seven >= 1:
        gen_comb(two, three, seven-1, path + [7], max_len, res)
    if two >= 1 and three >= 1:
        gen_comb(two-1, three-1, seven, path + [6], max_len, res)
    if two >= 2:
        gen_comb(two-2, three, seven, path + [4], max_len, res)
    if three >= 1:
        gen_comb(two, three-1, seven, path + [3], max_len, res)
    if two >= 1:
        gen_comb(two-1, three, seven, path + [2], max_len, res)


final_x = []
max_digits = 20
for candidate in candidates:
    b = candidate[0]
    b_str = str(b)
    b_len = len(b_str)
    two = candidate[2][2]
    three = candidate[2][3]
    seven = candidate[2][7]
    max_len = max_digits - b_len
    res = []
    gen_comb(two, three, seven, [], max_len, res)
    res = set(tuple(sorted(x)) for x in res)
    for comb in res:
        while len(comb) + b_len < max_digits:
            permutations = set(itertools.permutations(comb))
            for perm in permutations:
                perm = [str(x) for x in perm]
                a = "".join(perm)
                x = a + b_str
                x_set = set(x)
                if '1' in x_set and '2' in x_set and '3' in x_set and '4' in x_set and '6' in x_set and '7' in x_set and '8' in x_set and '9' in x_set:
                    if int(x) % b == 0:
                        final_x.append(x)
            comb += tuple([1])

In [3]:
int(sorted(final_x)[0])

1817198712146313216

In [4]:
candidates = []

ugly_numbers = [(72576, {2: 7, 3: 4, 7: 1})]  # Starting with 72576 and its factorization
index_2, index_3, index_7 = 0, 0, 0

# Initial multiples based on the first ugly number
next_2 = (ugly_numbers[0][0] * 2, {**ugly_numbers[0][1], 2: ugly_numbers[0][1][2] + 1})
next_3 = (ugly_numbers[0][0] * 3, {**ugly_numbers[0][1], 3: ugly_numbers[0][1][3] + 1})
next_7 = (ugly_numbers[0][0] * 7, {**ugly_numbers[0][1], 7: ugly_numbers[0][1][7] + 1})

# for _ in range(1, n):
while len(candidates) < 110:
    # Determine the minimum next ugly number
    next_ugly, factors = min([next_2, next_3, next_7], key=lambda x: x[0])

    # Append the new ugly number and its factors
    ugly_numbers.append((next_ugly, factors))
    s = Counter(str(next_ugly))
    if not ('0' in s or '5' in s):
        remaining = factors.copy()
        not_candidate = False
        for key, val in s.items():
            if key == '2':
                remaining[2] -= val
            elif key == '3':
                remaining[3] -= val
            elif key == '4':
                remaining[2] -= 2 * val
            elif key == '6':
                remaining[2] -= val
                remaining[3] -= val
            elif key == '7':
                remaining[7] -= val
            elif key == '8':
                remaining[2] -= 3 * val
            elif key == '9':
                remaining[3] -= 2 * val
        if remaining[2] < 0 or remaining[3] < 0 or remaining[7]< 0:
            not_candidate = True
        if not not_candidate:
            candidates.append((next_ugly, factors, remaining))

    # Update the pointers and calculate the next values
    if next_ugly == next_2[0]:
        index_2 += 1
        base_num, base_factors = ugly_numbers[index_2]
        next_2 = (base_num * 2, {**base_factors, 2: base_factors.get(2, 0) + 1})

    if next_ugly == next_3[0]:
        index_3 += 1
        base_num, base_factors = ugly_numbers[index_3]
        next_3 = (base_num * 3, {**base_factors, 3: base_factors.get(3, 0) + 1})

    if next_ugly == next_7[0]:
        index_7 += 1
        base_num, base_factors = ugly_numbers[index_7]
        next_7 = (base_num * 7, {**base_factors, 7: base_factors.get(7, 0) + 1})

In [5]:
def is_perfect_cube(n):
    cbrt = round(n ** (1/3))  # Calculate the cube root and round it to the nearest whole number
    return cbrt**3 == n       # Check if the cube of the rounded root equals the original number

heap = []
max_digits = 23

for candidate in candidates:
    b = candidate[0]
    b_str = str(b)
    b_len = len(b_str)
    two = candidate[2][2]
    three = candidate[2][3]
    seven = candidate[2][7]
    max_len = max_digits - b_len
    res = []
    gen_comb(two, three, seven, [], max_len, res)
    res = sorted(set(tuple(sorted(x)) for x in res))
    for comb in res:
        if len(comb) + b_len >= max_digits:
            break
        if is_perfect_cube(math.prod(comb)):
            a = "".join([str(x) for x in comb])
            x = a + b_str
            x_set = set(x)
            if '2' in x_set and '3' in x_set and '4' in x_set and '6' in x_set and '7' in x_set and '8' in x_set and '9' in x_set:
                if '1' not in x_set:
                    comb = tuple([1]) + comb
                while len(comb) + b_len < max_digits:
                    heapq.heappush(heap, (len(comb) + b_len, b, comb))
                    comb = tuple([1]) + comb

In [6]:
def permute_unique(elements):
    def backtrack(path, counter):
        if len(path) == len(elements):
            result.append(path[:])
            return
        for num in counter:
            if counter[num] > 0:
                path.append(num)
                counter[num] -= 1
                backtrack(path, counter)
                path.pop()
                counter[num] += 1

    result = []
    # Build a counter for the elements
    counter = {}
    for num in elements:
        if num in counter:
            counter[num] += 1
        else:
            counter[num] = 1
    backtrack([], counter)
    return result

found_len = float('inf')
res = []
while heap:
    comb_len, b, comb = heapq.heappop(heap)
    if comb_len > found_len:
        break
    b_str = str(b)
    permutations = permute_unique(comb)
    for perm in permutations:
        perm = [str(x) for x in perm]
        a = "".join(perm)
        x = a + b_str
        if res and int(x) > res[0]:
            break
        if int(x) % b == 0:
            found_len = len(x)
            res.append(int(x))
            break

sorted(res)[0]

8419411123272236924928